In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_2/models/B5R2b5_5FP_1FC_fold2_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold2.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(657, 25)
(657, 25)
Normal:  (429, 25)
Abnormal:  (228, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,667,667,8,P4,P42,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,57.0,114.0,659.0,583.0,0.059041,0.126961,0.678967,0.649402,AB01 P4-2 C008.JPG,AB01 P4-2 C008.JPG
1,1,1,668,668,8,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,130.0,133.0,614.0,476.0,0.131769,0.148873,0.619134,0.529880,AB01 P1 C008.JPG,AB01 P1 C008.JPG
2,2,2,669,669,8,P4,P41,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,227.0,135.0,611.0,441.0,0.225577,0.150865,0.605684,0.490040,AB01 P4-1 C008.JPG,AB01 P4-1 C008.JPG
3,3,3,670,670,8,P3,P31,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,60.0,177.0,596.0,510.0,0.062385,0.196682,0.611009,0.567729,AB01 P3-1 C008.JPG,AB01 P3-1 C008.JPG
4,4,4,671,671,8,P5,P51,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,184.0,141.0,478.0,473.0,0.184588,0.156841,0.478495,0.525896,AB01 P5-1 C008.JPG,AB01 P5-1 C008.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Views',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 657 validated image filenames belonging to 5 classes.
{0: 'FP-A', 1: 'FP-B', 2: 'FP-', 3: 'FP-D', 4: 'FP-E'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.17852098, -0.18951242, -0.13601738, ...,  0.08334167,
         0.21751319, -0.04067848],
       [ 0.90893078,  0.75860298,  0.29887679, ..., -0.12017911,
        -0.20001327,  0.53713435],
       [ 0.19715844,  0.27275708, -0.12705381, ..., -0.13861598,
        -0.09125907, -0.10467485],
       ...,
       [-0.19818996, -0.17831007,  0.26216039, ...,  0.65098435,
         0.55258286,  0.3497203 ],
       [-0.19998932, -0.19032015, -0.044836  , ...,  0.25639904,
         0.54455251,  0.4623726 ],
       [-0.2143168 , -0.22671928, -0.21641664, ...,  0.32131389,
        -0.23354323,  0.09808313]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.178521,-0.189512,-0.136017,0.047096,-0.028748,-0.160612,-0.117793,0.475820,0.172499,0.102105,...,0.031089,0.069715,-0.070849,0.032061,-0.108107,-0.071539,-0.129995,0.083342,0.217513,-0.040678
1,0.908931,0.758603,0.298877,0.287890,0.403891,0.088307,1.053717,0.444244,-0.100523,0.282498,...,-0.168331,-0.201594,-0.214941,-0.175971,-0.202177,0.722947,0.582893,-0.120179,-0.200013,0.537134
2,0.197158,0.272757,-0.127054,-0.122109,0.011950,0.258903,0.222708,0.094733,-0.095447,0.687709,...,0.108502,-0.176868,-0.137016,0.013067,-0.214800,0.386229,0.070162,-0.138616,-0.091259,-0.104675
3,-0.077483,-0.165508,-0.196963,0.153482,0.256192,0.167568,-0.046867,0.566151,0.250754,0.433602,...,0.355397,-0.165375,0.281389,0.445038,-0.188620,0.030919,-0.045154,0.411988,0.107962,-0.067503
4,-0.142057,-0.188177,-0.031904,0.182742,-0.174801,-0.084401,-0.016515,-0.122182,0.168869,-0.111820,...,0.533408,0.235359,0.244626,0.090197,0.169720,-0.030829,-0.191258,0.066340,0.163430,0.428316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,0.105989,0.265753,-0.172671,-0.104267,0.303567,0.097387,0.093854,0.160629,-0.073106,0.650188,...,-0.061708,-0.178716,-0.052535,0.126360,-0.205835,0.591393,0.279598,-0.138913,-0.119941,-0.132193
653,-0.128718,-0.057865,-0.029232,0.067478,0.312322,0.284825,-0.123920,0.583409,0.403349,0.808890,...,0.391363,-0.084232,0.283584,0.343811,-0.161914,0.293592,-0.115082,0.088262,0.456304,-0.055184
654,-0.198190,-0.178310,0.262160,0.631322,-0.163738,-0.203283,-0.119825,0.101738,0.883707,-0.229868,...,0.683949,1.065622,0.676073,0.082865,0.599576,-0.183784,-0.202042,0.650984,0.552583,0.349720
655,-0.199989,-0.190320,-0.044836,0.591421,-0.095140,-0.216883,-0.158195,0.138997,0.546067,0.036545,...,0.323746,0.502358,0.580778,0.253252,0.708941,-0.185347,-0.202922,0.256399,0.544553,0.462373


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.178521,-0.189512,-0.136017,0.047096,-0.028748,-0.160612,-0.117793,0.475820,0.172499,0.102105,...,-0.108107,-0.071539,-0.129995,0.083342,0.217513,-0.040678,Abnormal,P42,FP-B,AB01
1,0.908931,0.758603,0.298877,0.287890,0.403891,0.088307,1.053717,0.444244,-0.100523,0.282498,...,-0.202177,0.722947,0.582893,-0.120179,-0.200013,0.537134,Abnormal,P1,FP-A,AB01
2,0.197158,0.272757,-0.127054,-0.122109,0.011950,0.258903,0.222708,0.094733,-0.095447,0.687709,...,-0.214800,0.386229,0.070162,-0.138616,-0.091259,-0.104675,Abnormal,P41,FP-B,AB01
3,-0.077483,-0.165508,-0.196963,0.153482,0.256192,0.167568,-0.046867,0.566151,0.250754,0.433602,...,-0.188620,0.030919,-0.045154,0.411988,0.107962,-0.067503,Abnormal,P31,FP-B,AB01
4,-0.142057,-0.188177,-0.031904,0.182742,-0.174801,-0.084401,-0.016515,-0.122182,0.168869,-0.111820,...,0.169720,-0.030829,-0.191258,0.066340,0.163430,0.428316,Abnormal,P51,FP-C,AB01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,0.105989,0.265753,-0.172671,-0.104267,0.303567,0.097387,0.093854,0.160629,-0.073106,0.650188,...,-0.205835,0.591393,0.279598,-0.138913,-0.119941,-0.132193,Normal,P32,FP-B,Normal
653,-0.128718,-0.057865,-0.029232,0.067478,0.312322,0.284825,-0.123920,0.583409,0.403349,0.808890,...,-0.161914,0.293592,-0.115082,0.088262,0.456304,-0.055184,Normal,P42,FP-B,Normal
654,-0.198190,-0.178310,0.262160,0.631322,-0.163738,-0.203283,-0.119825,0.101738,0.883707,-0.229868,...,0.599576,-0.183784,-0.202042,0.650984,0.552583,0.349720,Normal,P52,FP-C,Normal
655,-0.199989,-0.190320,-0.044836,0.591421,-0.095140,-0.216883,-0.158195,0.138997,0.546067,0.036545,...,0.708941,-0.185347,-0.202922,0.256399,0.544553,0.462373,Normal,P61,FP-C,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_5FP_RF_5FP_fold2_1.csv')